In [1]:
import seaborn as sns
import string
import re
from collections import Counter
import matplotlib.pyplot as plt
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser','tagger','ner'])
import pandas as pd

In [2]:
import string
punctuations = string.punctuation
punctuations=punctuations+'—'+'–'+'--'

In [3]:
# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(text)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ !=
                "-PRON-" else word.lower_ for word in mytokens]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words]
    
    #Removing punctuations
    mytokens = [word for word in mytokens if word not in punctuations]

    mytokens = ' '.join(mytokens)
    # return preprocessed list of tokens
    return mytokens

In [4]:
campaign_speeches=pd.read_csv('Campaign Speeches')

In [5]:
campaign_speeches.head()

,candidate,date,speech title,text,party,won_nomination,sentiment_score,positive,negative
0,Robert Dole,1995-04-10,Remarks Announcing Candidacy for the Republica...,"Thank you, Governor Graves, Senator Kassebaum....",Republican,1,"{'neg': 0.088, 'neu': 0.736, 'pos': 0.175, 'co...",0.175,0.088
1,Robert Dole,1995-05-31,"Remarks in Los Angeles: ""Hollywood Speech""",I want to talk about a specific matter tonight...,Republican,1,"{'neg': 0.143, 'neu': 0.657, 'pos': 0.2, 'comp...",0.200,0.143
2,Robert Dole,1995-06-17,Remarks to the United States Conference of May...,"Thank you, Senator Kempthorne, and thank you a...",Republican,1,"{'neg': 0.12, 'neu': 0.711, 'pos': 0.17, 'comp...",0.170,0.120
3,Robert Dole,1995-08-19,Remarks at the Iowa Straw Poll in Ames,"Thank you, Dan Quayle , for that great introdu...",Republican,1,"{'neg': 0.103, 'neu': 0.717, 'pos': 0.181, 'co...",0.181,0.103
4,Robert Dole,1995-09-04,Remarks at the American Legion Convention in I...,"Thank you, and thank you very much, Commander ...",Republican,1,"{'neg': 0.103, 'neu': 0.725, 'pos': 0.172, 'co...",0.172,0.103


In [6]:
campaign_speeches=campaign_speeches.dropna(subset=['text'])

In [7]:
campaign_speeches.shape

(1691, 9)

In [9]:
party_speeches = campaign_speeches[(campaign_speeches.party == 'Democrat') & (campaign_speeches.won_nomination == 1)].reset_index(drop=True)

In [3]:
obama_speeches=pd.read_csv('Obama Speeches')

In [4]:
len(obama_speeches)

519

In [10]:
nlp.max_length=100000000

In [5]:
obama_text = [text for text in obama_speeches.text[0:100]]
obama_text = ' '.join(obama_text).strip()

In [92]:
f = open('obama.csv','w')
f.write(obama_text) #Give your csv text here.
## Python will convert \n to os.linesep
f.close()

In [54]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in punctuations]

In [55]:
tokens = separate_punc(obama_text)

In [56]:
tokens

['on',
 'behalf',
 'of',
 'the',
 'great',
 'state',
 'of',
 'illinois',
 'crossroads',
 'of',
 'a',
 'nation',
 'land',
 'of',
 'lincoln',
 'let',
 'me',
 'express',
 'my',
 'deep',
 'gratitude',
 'for',
 'the',
 'privilege',
 'of',
 'addressing',
 'this',
 'convention',
 'tonight',
 'is',
 'a',
 'particular',
 'honor',
 'for',
 'me',
 'because',
 'let',
 "'s",
 'face',
 'it',
 'my',
 'presence',
 'on',
 'this',
 'stage',
 'is',
 'pretty',
 'unlikely',
 'my',
 'father',
 'was',
 'a',
 'foreign',
 'student',
 'born',
 'and',
 'raised',
 'in',
 'a',
 'small',
 'village',
 'in',
 'kenya',
 'he',
 'grew',
 'up',
 'herding',
 'goats',
 'went',
 'to',
 'school',
 'in',
 'a',
 'tin',
 'roof',
 'shack',
 'his',
 'father',
 'my',
 'grandfather',
 'was',
 'a',
 'cook',
 'a',
 'domestic',
 'servant',
 'but',
 'my',
 'grandfather',
 'had',
 'larger',
 'dreams',
 'for',
 'his',
 'son',
 'through',
 'hard',
 'work',
 'and',
 'perseverance',
 'my',
 'father',
 'got',
 'a',
 'scholarship',
 'to',
 's

In [57]:
# organize into sequences of tokens
train_len = 40+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [58]:
text_sequences

[['on',
  'behalf',
  'of',
  'the',
  'great',
  'state',
  'of',
  'illinois',
  'crossroads',
  'of',
  'a',
  'nation',
  'land',
  'of',
  'lincoln',
  'let',
  'me',
  'express',
  'my',
  'deep',
  'gratitude',
  'for',
  'the',
  'privilege',
  'of',
  'addressing',
  'this',
  'convention',
  'tonight',
  'is',
  'a',
  'particular',
  'honor',
  'for',
  'me',
  'because',
  'let',
  "'s",
  'face',
  'it',
  'my'],
 ['behalf',
  'of',
  'the',
  'great',
  'state',
  'of',
  'illinois',
  'crossroads',
  'of',
  'a',
  'nation',
  'land',
  'of',
  'lincoln',
  'let',
  'me',
  'express',
  'my',
  'deep',
  'gratitude',
  'for',
  'the',
  'privilege',
  'of',
  'addressing',
  'this',
  'convention',
  'tonight',
  'is',
  'a',
  'particular',
  'honor',
  'for',
  'me',
  'because',
  'let',
  "'s",
  'face',
  'it',
  'my',
  'presence'],
 ['of',
  'the',
  'great',
  'state',
  'of',
  'illinois',
  'crossroads',
  'of',
  'a',
  'nation',
  'land',
  'of',
  'lincoln',

In [18]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [59]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [60]:
sequences[0]

[21,
 1385,
 5,
 1,
 204,
 227,
 5,
 427,
 4395,
 5,
 6,
 165,
 968,
 5,
 1795,
 133,
 76,
 2106,
 48,
 1885,
 5342,
 13,
 1,
 2558,
 5,
 3412,
 15,
 1336,
 561,
 11,
 6,
 1643,
 826,
 13,
 76,
 44,
 133,
 10,
 239,
 12,
 48]

In [61]:
tokenizer.index_word

{1: 'the',
 2: 'and',
 3: 'to',
 4: 'that',
 5: 'of',
 6: 'a',
 7: 'we',
 8: 'in',
 9: 'i',
 10: "'s",
 11: 'is',
 12: 'it',
 13: 'for',
 14: 'you',
 15: 'this',
 16: 'our',
 17: 'have',
 18: 'are',
 19: 'not',
 20: "n't",
 21: 'on',
 22: 'they',
 23: 'who',
 24: 'but',
 25: 'will',
 26: 'be',
 27: 'do',
 28: 'with',
 29: 'what',
 30: 'as',
 31: 'can',
 32: 'when',
 33: 'was',
 34: 'people',
 35: 'about',
 36: 'their',
 37: 'from',
 38: 'or',
 39: 'all',
 40: 'more',
 41: 'so',
 42: 'by',
 43: 'know',
 44: 'because',
 45: 'there',
 46: "'ve",
 47: 'if',
 48: 'my',
 49: 'has',
 50: 'at',
 51: "'re",
 52: 'america',
 53: 'us',
 54: 'time',
 55: 'he',
 56: 'up',
 57: 'been',
 58: 'an',
 59: 'country',
 60: 'american',
 61: 'president',
 62: 'think',
 63: 'just',
 64: 'one',
 65: 'out',
 66: 'going',
 67: 'war',
 68: 'them',
 69: 'obama',
 70: 'make',
 71: 'those',
 72: 'need',
 73: 'new',
 74: 'your',
 75: 'now',
 76: 'me',
 77: 'would',
 78: 'how',
 79: 'change',
 80: 'iraq',
 81: 'some'

In [22]:
tokenizer.word_counts

OrderedDict([('on', 49193),
             ('behalf', 522),
             ('of', 187076),
             ('the', 354324),
             ('great', 4945),
             ('state', 4530),
             ('illinois', 2348),
             ('crossroads', 87),
             ('a', 161991),
             ('nation', 6252),
             ('land', 871),
             ('lincoln', 379),
             ('let', 8128),
             ('me', 15071),
             ('express', 304),
             ('my', 23913),
             ('deep', 358),
             ('gratitude', 73),
             ('for', 85016),
             ('privilege', 232),
             ('addressing', 156),
             ('this', 70119),
             ('convention', 572),
             ('tonight', 1664),
             ('is', 88548),
             ('particular', 442),
             ('honor', 1092),
             ('because', 24362),
             ("'s", 98800),
             ('face', 4264),
             ('it', 87542),
             ('presence', 364),
             ('stage', 598),
 

In [62]:
vocabulary_size = len(tokenizer.word_counts)

In [63]:
vocabulary_size

10631

### Convert to Numpy Matrix


In [64]:
import numpy as np

In [65]:
sequences = np.array(sequences)

In [66]:
sequences

array([[   21,  1385,     5, ...,   239,    12,    48],
       [ 1385,     5,     1, ...,    12,    48,  1796],
       [    5,     1,   204, ...,    48,  1796,    21],
       ...,
       [   16,  2000,  1020, ...,  2107,     1,   152],
       [ 2000,  1020,    17, ...,     1,   152,   124],
       [ 1020,    17, 10631, ...,   152,   124,     5]])

# Creating an LSTM based model


In [67]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.layers import Dropout

In [81]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 40, input_length=seq_len))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
   
    model.summary()
    
    return model

### Train / Test Split

In [69]:
from keras.utils import to_categorical


In [70]:
sequences

array([[   21,  1385,     5, ...,   239,    12,    48],
       [ 1385,     5,     1, ...,    12,    48,  1796],
       [    5,     1,   204, ...,    48,  1796,    21],
       ...,
       [   16,  2000,  1020, ...,  2107,     1,   152],
       [ 2000,  1020,    17, ...,     1,   152,   124],
       [ 1020,    17, 10631, ...,   152,   124,     5]])

In [71]:
# First 49 words
sequences[:,:-1]

array([[   21,  1385,     5, ...,    10,   239,    12],
       [ 1385,     5,     1, ...,   239,    12,    48],
       [    5,     1,   204, ...,    12,    48,  1796],
       ...,
       [   16,  2000,  1020, ...,   426,  2107,     1],
       [ 2000,  1020,    17, ...,  2107,     1,   152],
       [ 1020,    17, 10631, ...,     1,   152,   124]])

In [72]:
# last Word
sequences[:,-1]

array([  48, 1796,   21, ...,  152,  124,    5])

In [73]:
X = sequences[:,:-1]

In [74]:
y = sequences[:,-1]

In [75]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [76]:
seq_len = X.shape[1]

In [77]:
seq_len

40

In [82]:
# define model
model = create_model(vocabulary_size+1, seq_len)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 40)            425280    
_________________________________________________________________
lstm_7 (LSTM)                (None, 40, 128)           86528     
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10632)             1371528   
Total params: 2,031,432
Tra

In [83]:
from pickle import dump,load

In [84]:
X.shape

(303533, 40)

In [85]:
from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [86]:
# fit model
results=model.fit(X, y, batch_size=1024, epochs=150,verbose=1,callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
209920/303533 [===================>..........] - ETA: 30s - loss: 6.7800 - categorical_accuracy: 0.0411

KeyboardInterrupt: 

In [ ]:
model.save('model1.h5')

In [ ]:
# save the tokenizer
dump(tokenizer, open('my_tokenizer', 'wb'))

In [ ]:
hist = results.history
loss_values = hist['loss']
acc_values = hist['categorical_accuracy'] 

epochs = range(1, len(loss_values) + 1)

plt.figure(figsize=(15,4))
plt.subplot(121)
plt.plot(epochs, loss_values, 'g.', label='Loss')

plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(122)
plt.plot(epochs, acc_values, 'r.', label='Training acc')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [2]:
from textgenrnn import textgenrnn


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 15,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [88]:
len(obama_text)

1649213

In [7]:
file_name = 'obama.csv'
model_name = 'Obama'   # change to set file name of resulting trained models/texts

In [12]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 4-layer, 128-cell LSTMs
Training on 1,319,151 character sequences.
Epoch 1/15
1288/1288 [==============================] - 219s 170ms/step - loss: 3.0638 - val_loss: 3.0551
Epoch 2/15
1288/1288 [==============================] - 222s 172ms/step - loss: 2.4571 - val_loss: 1.5205
Epoch 3/15
1288/1288 [==============================] - 221s 171ms/step - loss: 1.4221 - val_loss: 1.2763
Epoch 4/15
1288/1288 [==============================] - 222s 172ms/step - loss: 1.2700 - val_loss: 1.1976
Epoch 5/15
1288/1288 [==============================] - 221s 172ms/step - loss: 1.2001 - val_loss: 1.1583
####################
Temperature: 0.2
####################
nd a standard and the policy of the problems and the strategy in the world. But what I have to make it is that the problem is that the programs are the country that we have to make sure that the problem is that the change of the world has been a both in the problem. It's time to turn the page on the

 do it to stand up t

he last new century affordable for all Americans of the war in Iraq and a state where people who can simply come together and stand up and the other day. We have to get the challenges we face are willing to take the benefits of the war in Iraq, but some of these kids are willing to see them from the

 failed to make their home to take the same choice in this country - or the same thing that we have to recognize that we have a chance to stay in the special interest in the streets. And I will make sure that the problem is the basic standard of the strategic policies of lobbyists that we can start 

e same of the most students who are the strength of this campaign – that says that we can still take a chance to start working at the country to give them the future of the reason to be a plan that will fight for the problem when we were written at the last seven years. The special interests of the 

####################
Temperature: 1.0
####################
ve always knew, I think it'g not be

In [3]:
file_name = 'obama.csv'
model_name = 'Obama1'   # change to set file name of resulting trained models/texts

In [4]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 150,   # set higher to train the model for longer
    'gen_epochs': 3,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell LSTMs
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 279,838 word sequences.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/150
273/273 [==============================] - 43s 156ms/step - loss: 6.3593
Epoch 2/150
273/273 [==============================] - 39s 143ms/step - loss: 6.0542
Epoch 3/150
273/273 [==============================] - 39s 142ms/step - loss: 4.8596
####################
Temperature: 0.2
####################
afford to be able to be able to take the same way that we ' re going to be able to be able to take a better tax cut that will be a family that ' s harder to be a sister who ' s been a sense of the united states . we have to make a difference in our country . we ' re going to play offense to play offense for the same kind of american people . and i ' ll also kno

of approach will be one school is tax black every college . and i ' ll keep my family to take the low - cradle to the sources of working americans who will be easy these differences spent . we have to accept that they new our world like so many system in our military 100 of troops - it ' s just hope in service before how many man suggested that i ' m phd , , you take them independents , who ' s not going to the crumbling practice , the extensively for brought the lobbying medical salary , the government must improve the failures of people are like but we have to take out of the situation we military on a sense of violence to the nominee . ( cheers , applause . ) is . and , applause . we have to have a way in this speech the kind of transition that christ clear on so many of black , all in a constitution legacy of the victims of america and families across long - neighborhoods . finally debating this party i ' t didn ' t ask for these tough hoped . well , in this debate , it ' s been a 

establish a universal health care plan that will cover every american a vehicle to the same kind of politics we need . and i won ' t just talk about the future of this generation . and when i am president , i will create a 21st century va to prevent the 21st century - american seek to meet the world . but there are no surprises . it is a privilege to be contained . senator mccain is not in the white house , we ' ve had to be focused on the long time . but we ' re going to talk about it . we ' ve got to talk about our values and our ideals . and we must also rebuild our standing on the streets of the americas . we ' ll have to keep pace the price of gas tax credits to the middle - class families . and i ' ll establish a new supportive services housing - - american community , and support , and the determination to be a part of american dream . we must have a party that runs prisons which lock an african - american community , and the non - of us to address the international community . 

273/273 [==============================] - 39s 143ms/step - loss: 3.3993
Epoch 8/150
273/273 [==============================] - 40s 145ms/step - loss: 3.1788
Epoch 9/150
273/273 [==============================] - 40s 148ms/step - loss: 2.9929
####################
Temperature: 0.2
####################
out . we ' re going to have to make sure that the american people are still the skills they need . and it ' s time to stop the fight for justice and bring the world ' s most dangerous weapons . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page on the arrogance and distractions and distraction , and agrees with the united states of america . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it '

to find and common security throughout america . i don ' t want fema to be trade here now and to begin this back to this country . russert : right . . . . . obama : right , even . russert : when you may return " the first area in seeing medical students so students records work with service , commander - your support - - - if you replace you are willing to be getting sex essentially , but they ' re coming at the politics of the bargain . ( begin video clip ) unidentified male : i ' m sure , not only thought matthews : some change is a resolution to edwards at the same way that those consequences have skyrocketed you have interpreted to get new textbooks or the pension he made in persuade . when i came back to senator clinton , i found it because of some of those of these americans are being absolutely long as main street . so there are in the next election . we all have the technology we face on tv , as president of the united states senate will just be about where we might be less tha

...

273/273 [==============================] - 39s 144ms/step - loss: 0.5088
####################
Temperature: 0.2
####################
the things that really voters about who ' s called in my own ? " obama : well , you know about the democrats that have made us over time . we still know that these who ' s born from the special interests , and we know that this is the kind of politics you will love . we ' ve seen progress face of the reality of care and opportunity for america for america and the children to be end . this will be a moment when we started up and republicans in the time . we don ' t have to tell you what we are doing , what is we want to be about my challenge is to the moment in the president . it ' s been hard to turn the page . it is time to write the politics of a new century . we are a nation that has led generations of weary travelers to win generations , and liberty , and hope . we know that this time is a responsibility to come together , and we are not a nation that 

and the mother can ' t get the chance to reach for their own , to ask for your standing , and for their rights and grandchildren . so we learned in the democratic country into a lot of time for american democrats and coming together forward , you ' re ready to turn the page on washington , and that ' s the america i ' m when we ' re working about to . we don ' t george washington is on us wants to go in up , who are young forgotten . they are working harder to be . today should be a hand for it . it is neighborhoods we ' ve lost - that when it ' s hard about the world when it was , the other best administration was that was young people to take on a path and their families , fighting to washington . it ' s with time . it ' s how we ' ve already in that moment . it ' s time for new leadership to make sure that our kids are safe - ever and opportunity ' s past , we put the jobs back the most of washington - then we ' ll make sure that all the american workers start being mine , what it d

is the way that we are still trying to heed the call of the united states . roberts : oh , you have really been pounded by the clinton trail . i know that bodes well for the november elections . stephanopoulos : so you believe the ideals will come behind , i said , " we all have an administration in america . it ' s about hard china that this generation has come up before our country want to walk away , ' to do their . . . ( laughter ) obama : . . . and that ' s the point that ' s within our grasp . so i ' m asking you to march with me , and work , and tell the country that stand before we before that you still have a politics that ' s been trying to drive with our troops , and we ' ve got to have some arguments . russert : but the time you face in washington , the and most game of them are being in the region . obama : well , i ' m not going to continue sort of the tit - off - tat . that is where i passed a first plan in the senate that will ' ve do , not the only - that act has been 

273/273 [==============================] - 38s 140ms/step - loss: 0.4724
Epoch 105/150
273/273 [==============================] - 39s 144ms/step - loss: 0.4664
####################
Temperature: 0.2
####################
stay in a policy of opportunity and bring this country back together and to ask you to the south , where you ' re on your own but life and your ability to tell the story together , and who will make sure that your government will be willing to make sure that every american has comes to end the war in iraq . i am running because the american moves that we will come to make progress will compete with her nations ' s bills . we know that accountability here is . it is the fact that led the education of the gulf coast as a decent result in the region . when i am president , we cannot finish the counter - out of our power and carrying the values of the world . it ' s about whether we settle to see the next president . we have to do more . and they know we can all those who ha

time , john mccain and hillary clinton also the people who do . and i believe that together is who that we don ' t have to wait in the world , that those ideals are challenging the chance to pass the power that will do not part , but there have never been in washington . russert : let me ask you , but let them just say that this is this job . i ' ve seen in the courts as a week , an living wage , how you will make that future . let ' s be the generation that that doesn ' t offer when you need to take look of this war in iraq as well . but i ' m the only candidate . i do not want the world - to make real its own political , and in twenty - years say , " or when the years of hope - that this country will be different . this is what we ' ve been in getting when i get making like the money of the first time since the most . time had a better life in their future . we can transform our cause and our cynicism and after the world ' s greatest threat . it ' s time for a foreign policy that say

In [ ]:
file_name = 'democratic_nominees_text.csv'
model_name = 'democratic_nominee'   # change to set file name of resulting trained models/texts

In [ ]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 150,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])